In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import sys
import nltk
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import metrics

In [ ]:
df = pd.read_excel('/Users/anamolpundle/Documents/Insight/NOS_database_table_of_contents.xls', sheetname='phase1_table_of_contents')

In [ ]:
""" List of contested authors"""

author_list = ['Greene, Robert', 'Marlowe, Christopher', 'Shakespeare', 
               'Peele, George', 'Kyd, Thomas', 
               'Nash, Thomas', 'Watson, Thomas', 'Munday', 'Lodge', 
               'Lyly', 'Drayton', 'Achelley', 'Chettle'] 


In [ ]:
""" Replace Shakespeare by 'good' plays only """

all_texts = []
for ctr, author in enumerate(author_list):
    if (author == 'Shakespeare'):
        author = 'Shakesgood'
    author = author.replace(" ", "")
    path = '/Users/anamolpundle/Documents/Insight/MLDatatxt/' + author + '/'
    all_author = ''
    for file in os.listdir(path):
        text = open(path + file, 'r').read()
        all_author = all_author + text
    all_texts.append(all_author)

In [ ]:
""" Split text into documents of 3000 words each """


words_per_doc = 3000
all_docs = []

for ctr, text in enumerate(all_texts):
    text_words = nltk.tokenize.word_tokenize(text)
    docs = []
    
    for i in range(0,len(text_words),words_per_doc):
        if (i + words_per_doc >= len(text_words)):
            docs.append(" ".join(text_words[i:]))
        else:
            docs.append(" ".join(text_words[i:i+words_per_doc]))
    docs[-2] = docs[-1] + docs[-2]
    del docs[-1]
    all_docs.append(docs)
        

In [ ]:
""" Save authors into author_array and texts into text_array"""
""" Remove Achelly, Kyd and Chettle"""

author_array = []
text_array = []
all_text = ""
for ctr, author in enumerate(all_docs):
    if ((author_list[ctr] != 'Achelley') and (author_list[ctr] !='Chettle') and (author_list[ctr] !='Kyd, Thomas')):
        for text in author:
            author_array.append(author_list[ctr])
            text_array.append(text)

In [ ]:
""" Split into train and holdout data sets. Imbalance not corrected """

X_train, X_holdout, y_train, y_holdout = train_test_split(text_array, author_array, test_size = 0.15, random_state = 42)

In [ ]:
"""Split train into train and validation set. Imbalance not corrected"""

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [ ]:
all_train_text_data = ""
for doc in X_train:
    all_train_text_data = all_train_text_data + ' ' + doc

In [ ]:
""" TFIDF instead of bag of words. Takes a while (~10 mins) """

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

X_train_no_digits = ''.join([i for i in all_train_text_data if not i.isdigit()])
all_words = tokenize(X_train_no_digits)

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=3 , max_df = 0.5, ngram_range=(1,4), max_features = 400000)

train_tfs = tfidf.fit_transform(X_train)
test_tfs = tfidf.transform(X_test)
holdout_tfs = tfidf.transform(X_holdout)

X_train_df = pd.DataFrame(train_tfs.toarray())
X_test_df = pd.DataFrame(test_tfs.toarray())
X_holdout_df = pd.DataFrame(holdout_tfs.toarray())

In [ ]:
pipeline.